In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import xml.etree.ElementTree as et 
print(pd.get_option("display.max_rows"), pd.get_option("display.max_columns"), pd.get_option("display.max_colwidth"))
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("display.max_colwidth", 200)

cwd = '/Volumes/GoogleDrive/Shared drives/AREA_SCI_2021/ZC_2021_JG_BATTELLE_PART2/'
cwd = '/Volumes/GoogleDrive/Shared drives/BATTELLE_2021_PART2/ZC_2021_JG_BATTELLE_PART2/'

## TROUBLESHOOT:

to add here....

according to study plan there are 41 codes: 

In [154]:
# all compounds we should have: 41 codes INFO from study plan
shouldhave = [1046, 1951, 1956, 1694, 1171, 1271, 1584, 1728, 1975, 1895
    , 1825, 1770, 1623, 1437, 1547, 1965, 1747, 1477, 1724, 1978
    , 1625, 1036, 1355, 1719, 1111, 1323, 1656, 1240, 1247, 1989
    , 1409, 1771, 1693, 1630, 1794, 1919, 1072, 1454, 1229, 1200, 1804]

comps = []
for i, item in enumerate(os.listdir(os.path.join(cwd, 'plate_csvs'))):
    c = item.split('_')[1]
    if item.split('_')[2] == 'DM': # only process DM experiments <----------------------------------------------
        comps.append(c)

missing = [c for c in shouldhave if str(c) not in comps]
print('missing compounds:\n', missing)

# tell me if I don't have 3 replicates
cs = pd.Series(comps)
cs_counts = cs.value_counts()
for c, comp in enumerate(comps): 
    if cs_counts[comp] != 3:
        print('for compound:', comp, 'I find:', cs_counts[comp], 'replicates')

missing compounds:
 []


# ---- MUNCH -----

In [155]:
path = os.path.join(cwd, 'plate_csvs')
fils = os.listdir(path)
print(len(fils))

# only process DM experiments <----------------------------------------------
fils = [item for item in fils if '_DM_' in item]
print(len(fils))
df = pd.DataFrame(data=fils, columns=['filenames'])

df['date'] = df['filenames'].map(lambda x: x.split('_')[0])
df['compound'] = df['filenames'].map(lambda x: x.split('_')[1])
df['condition'] = df['filenames'].map(lambda x: x.split('_')[2])
df['replicate'] = df['filenames'].map(lambda x: x.split('.csv')[0].split('_')[-1])

print(df.shape[0], 'experiments and ', df['compound'].unique().shape[0], 'compounds')

df_platefolders = df.copy()


246
123
123 experiments and  41 compounds


## get compounds for which x conditions and for each condition 3 replicates

In [156]:
g = df.groupby(['compound', 'condition'])['replicate'].count()
g = g[g == 3] # 3 replicates per condition
g.head()

compound  condition
1036      DM           3
1046      DM           3
1072      DM           3
1111      DM           3
1171      DM           3
Name: replicate, dtype: int64

In [157]:
c = g.index.get_level_values(0).to_numpy()
unique, counts = np.unique(c, return_counts=True)
c = list(zip(unique, counts))

def n_conditions(c): 
    n_cond = 1 # one condition
    if c[1] == n_cond: 
        return True
    
c = filter(n_conditions, c) # two conditions 
c2 = [item[0] for item in list(c)]

# add also pos ctrls - if there are plates with only pos control
# c2 = ['0000']+c2
print(len(c2), 'compunds meet criteria : ',c2)
df_platefolders2 = df_platefolders[df_platefolders['compound'].isin(c2)].copy()
df_platefolders2.shape

41 compunds meet criteria :  ['1036', '1046', '1072', '1111', '1171', '1200', '1229', '1240', '1247', '1271', '1323', '1355', '1409', '1437', '1454', '1477', '1547', '1584', '1623', '1625', '1630', '1656', '1693', '1694', '1719', '1724', '1728', '1747', '1770', '1771', '1794', '1804', '1825', '1895', '1919', '1951', '1956', '1965', '1975', '1978', '1989']


(123, 5)

In [158]:
def parse_xml_full(xml_path):
    msg_img_well_names = [] # report if well name doesn't coincide with file name
    msg_corrected = '' # correction finished
    msg_phenotype = []
    msg_phenotype_finished = []
    
    booleans = ['bodycurvature', 'snoutjawdefects', 'yolkedema', 'necrosis', 'tailbending', 'notochorddefects'
            , 'craniofacialedema', 'finabsence', 'scoliosis']
    measured = ['fishoutline', 'eyes', 'pigmentation', 'heart', 'bodylength']
    
    '''check childrens status'''
    rows = []
    xtree = et.parse(xml_path)
    xroot = xtree.getroot() 
    plate_name = xroot.attrib.get("plate_name")
    wells = []
    correction = []
    
    for node in xroot: 
        well_name = node.attrib.get("name")
        wells.append(well_name)
        
    for well in wells: 
        thiswell = xroot.find(".//*[@name='"+well+"']")      
        # check if well folder and well name are correct
        wellfolder = thiswell.attrib.get('well_folder') 
        dor = thiswell.attrib.get('dorsal_image') 
        lat = thiswell.attrib.get('lateral_image') 
        if wellfolder not in ('None', None): 
            if well in wellfolder and well in dor and well in lat:
                pass
            else: 
                msg_img_well_names.append(' '.join([well, wellfolder, dor, lat]))

        # collect all done - all wells with that child should have status = 'finished'
        corrected = thiswell.attrib.get('correction_done')
        correction.append(corrected)
    if 'True' in correction: 
        pass
    else: msg_corrected = 'correction NOT finished'
        
    # check all phenotypes are there and finished - status is either None or 'finished'
    pt_status_dict = {} # phenotype : ['finished']    
    for m, mb in enumerate((booleans+measured)):
        status = []
        for well in wells: 
            thiswell = xroot.find(".//*[@name='"+well+"']")
            element = thiswell.find(mb)
            if element != None:
                status.append(element.get('status'))
        pt_status_dict[mb] = status
        
    for pt in (booleans+measured): 
        if pt in pt_status_dict.keys(): 
            pass
        else: 
            msg_phenotype.append(pt)
            
    for k in pt_status_dict.keys(): 
        if 'finished' in pt_status_dict[k]: 
            pass
        else: 
            msg_phenotype_finished.append(k)

    cpd_good2go = True
    
    if msg_img_well_names != []:
        cpd_good2go = False
        print('disprepancies of well & image names: '+' '.join(msg_img_well_names))
    if msg_corrected != '':
        cpd_good2go = False
        print('correction not finished --> '+msg_corrected)
    if msg_phenotype != []:
        cpd_good2go = False
        print('phenotypes missing:'+' '.join(msg_phenotype))
    if msg_phenotype_finished != []:
        cpd_good2go = False
        print('phenotypes not finished: '+' '.join(msg_phenotype_finished))
    return cpd_good2go

### mmmh do not like that - we are checking on the xml but aren't updating the csv.... 
# then reading the csv for analysis...

# TODO fix that absolutely!!

# ---- MUNCH -----

In [159]:
# get errors and select compounds to proceed
df_platefolders['good2go'] = np.nan

for c, compound in enumerate(pd.Series(comps).unique()):
    print('check:', c+1 , '--> ' ,compound)
    files = df_platefolders[df_platefolders['compound']==compound]['filenames']
    files.reset_index()
    for i in files.index: 
        print(i, files[i].split('.csv')[0])    
        f = files[i].split('.csv')[0]
        xml_path = os.path.join(cwd, 'VAST_images', f, f+'.xml')
        g2g = parse_xml_full(xml_path)
        df_platefolders.loc[df_platefolders['filenames']==files[i], 'good2go'] = g2g
        

check: 1 -->  1046
0 20220326_1046_DM_R1
5 20220328_1046_DM_R2
10 20220329_1046_DM_R3
check: 2 -->  1171
1 20220326_1171_DM_R1
6 20220328_1171_DM_R2
11 20220329_1171_DM_R3
check: 3 -->  1694
2 20220326_1694_DM_R1
7 20220328_1694_DM_R2
12 20220329_1694_DM_R3
check: 4 -->  1951
3 20220326_1951_DM_R1
8 20220328_1951_DM_R2
13 20220329_1951_DM_R3
check: 5 -->  1956
4 20220326_1956_DM_R1
9 20220328_1956_DM_R2
14 20220329_1956_DM_R3
check: 6 -->  1271
15 20220402_1271_DM_R1
20 20220404_1271_DM_R2
117 20220810_1271_DM_R6
check: 7 -->  1584
16 20220402_1584_DM_R1
21 20220404_1584_DM_R2
24 20220405_1584_DM_R3
check: 8 -->  1728
17 20220402_1728_DM_R1
22 20220404_1728_DM_R2
25 20220405_1728_DM_R3
check: 9 -->  1895
18 20220402_1895_DM_R1
23 20220404_1895_DM_R2
26 20220405_1895_DM_R3
check: 10 -->  1975
19 20220402_1975_DM_R1
27 20220405_1975_DM_R3
121 20221002_1975_DM_R4
check: 11 -->  1437
28 20220409_1437_DM_R1
33 20220411_1437_DM_R2
38 20220417_1437_DM_R4
check: 12 -->  1547
29 20220409_1547_D

In [160]:
#B1 =  ['1046', '1171', '1271', '1694', '1951', '1956', '1584', '1728', '1895', '1825']
notg2g = df_platefolders[df_platefolders['good2go']==False]['compound'].drop_duplicates().tolist()
print('notg2g', notg2g)

cpd2go = [cpd for cpd in df_platefolders['compound'].drop_duplicates().tolist() if cpd not in notg2g]
#
#cpd2go = [cpd for cpd in cpd2go if cpd not in B1+B2+['1804', '1794']]

print(len(cpd2go))
cpd2go.sort()
print(cpd2go, len(cpd2go))

notg2g []
41
['1036', '1046', '1072', '1111', '1171', '1200', '1229', '1240', '1247', '1271', '1323', '1355', '1409', '1437', '1454', '1477', '1547', '1584', '1623', '1625', '1630', '1656', '1693', '1694', '1719', '1724', '1728', '1747', '1770', '1771', '1794', '1804', '1825', '1895', '1919', '1951', '1956', '1965', '1975', '1978', '1989'] 41


# percentages for dose reponse curve

## first check concentrations
### there should be 7 concentrations (for compounds) with 66 entries - but decimals are sometimes different. 
### check also controls (how many conc for DMSO ???)

## TODO --> fix xmls & re-run


In [164]:
# first check concentrations
def check_conc_decimals(df1, compound):
    df = df1.copy()
    df  = df[df['compound'] == compound].copy()
    df['dose_unit'] = df['dose'].astype(str)+df['unit']
    g = df.groupby(['dose_unit'])['avg_eye_diameter'].size()
    #g = df.groupby(['dose'])['avg_eye_diameter'].size()
    if g.unique().shape[0] == 1: 
        pass
    else: 
        print('something wrong with concentrations\n', g)
    return g


c2 = [str(x) for x in cpd2go]
for c, compound in enumerate(c2):
    dfs = []
    print('compound:', c+1 , '--> ' ,compound)
    files = df[df['compound']==compound]['filenames']
    files.reset_index()
    for i in files.index: 
        #print(files[i])
        d = pd.read_csv(os.path.join(path, files[i]))
        d['plate_id'] = files[i].split('.csv')[0]
        dfs.append(d)
    alldfs = pd.concat(dfs)
    # check COMPOUND or DMSO or DCA # <-------------------- CHANGE HERE
    check_conc_decimals(alldfs, compound)
    # check_conc_decimals(alldfs, 'DMSO')
    # check_conc_decimals(alldfs, 'DCA')

compound: 1 -->  1036
compound: 2 -->  1046
compound: 3 -->  1072
compound: 4 -->  1111
compound: 5 -->  1171
compound: 6 -->  1200
compound: 7 -->  1229
compound: 8 -->  1240
compound: 9 -->  1247
compound: 10 -->  1271
compound: 11 -->  1323
compound: 12 -->  1355
compound: 13 -->  1409
compound: 14 -->  1437
compound: 15 -->  1454
compound: 16 -->  1477
compound: 17 -->  1547
compound: 18 -->  1584
compound: 19 -->  1623
compound: 20 -->  1625
compound: 21 -->  1630
compound: 22 -->  1656
compound: 23 -->  1693
compound: 24 -->  1694
compound: 25 -->  1719
compound: 26 -->  1724
compound: 27 -->  1728
compound: 28 -->  1747
compound: 29 -->  1770
compound: 30 -->  1771
compound: 31 -->  1794
compound: 32 -->  1804
compound: 33 -->  1825
compound: 34 -->  1895
compound: 35 -->  1919
compound: 36 -->  1951
compound: 37 -->  1956
compound: 38 -->  1965
compound: 39 -->  1975
compound: 40 -->  1978
compound: 41 -->  1989


## get percentages

### convert first!


The way that stuff was annotated is that embryos are set to **unhatched ONLY if they are alive**. 
So unhatched needs to go inside phenotypes - needs to be computed over the population of alive embryos.  



In [165]:
def df_format(df1):
    df = df1.copy()
    for col in df.columns: 
        df[col].replace([None,'None','NaN'],np.nan, inplace=True, regex=True)
        df[col].replace([False,'False'],0.0, inplace=True, regex=True)
        df[col].replace([True,'True'],1.0, inplace=True, regex=True)
    # check what else has to enter? dead120 where dead24 - seems it's already fine
    for col in df.columns: 
        pd.to_numeric(df[col], errors='ignore')
    return df


In [166]:
# get files for compound 
# FORMAT
# select one exposure, three replicates
# --> different for:
# i) image data 
# get percentage and nSamples ()
# ii) dead24 dead120 unhatched [dead24, dead120, unhatched120]
# get percentage and nSamples ()

booleans = ['bodycurvature', 'snoutjawdefects', 'yolkedema', 'necrosis', 'tailbending', 'notochorddefects'
            , 'craniofacialedema', 'finabsence', 'scoliosis']
measured_bool = ['Craniofacial: eye defects', 'Trunk: short', 'Trunk: long', 'Percardial tissue: Edema'
                 , 'Pigment: abnormal', 'Pigment: decreased'] # --> we don't measure 'Pigment: absent coloration']    

dead_unhatched = ['dead24', 'dead120', 'unhatched120']

def fetch_csvs_byCompound(compound, filesdf): 
    '''pass compound as string'''
    dfs = []
    files = filesdf[filesdf['compound']==compound]['filenames']
    files.reset_index()
    print('found files: \n', files)
    for i in files.index: 
        d = pd.read_csv(os.path.join(path, files[i]))
        d['plate_id'] = files[i].split('.csv')[0]
        dfs.append(d)
    alldfs = pd.concat(dfs)
    return alldfs

def select_by_exposure(df): 
    '''return list of dfs by exposure'''
    exposures = df['exposure'].unique()
    chorion = df['chorion'].unique()
    
    df['exposure_chorion'] = df['exposure']+'_'+df['chorion']
    exp_chor = df['exposure_chorion'].unique()
    dfs = []
    for e in exp_chor: 
        edf = df[df['exposure_chorion']==e].copy()
        dfs.append(edf)
    return dfs

def get_percentage_nSamples(df1, column):
    #print('debugging my columns: ', column, df1[column].unique()) # check what's inside booleans...
    df = df1.copy()
    s = df.groupby(['dose', 'plate_id']).apply(lambda x: float(x[column].sum()))
    c = df.groupby(['dose', 'plate_id']).apply(lambda x: float(x[column].count()))
    pct = s/c*100
    nSamples = c
    pct = pct.unstack(level=1)
    nSamples = nSamples.unstack(level=1)    
    pct.rename(columns=dict(zip(pct.columns, [col+' - '+column for col in pct.columns])), inplace=True)
    nSamples.rename(columns=dict(zip(nSamples.columns, [col+' - '+'nSamples' for col in nSamples.columns])), inplace=True)
    return pct, nSamples

def mort_dca(df1, column):
    # DCA
    d120 = df1.groupby(['dose', 'plate_id']).apply(lambda x: float(x[column].sum())).unstack(level=1)
    if not os.path.isfile(os.path.join(cwd, 'Percentages_4PRISM', 'DCA_'+column+'.csv')):
        print('no file')
        d120.to_csv(os.path.join(cwd, 'Percentages_4PRISM', 'DCA_'+column+'.csv'))
        #print('initial saving')
    else: 
        DCA_120 = pd.read_csv(os.path.join(cwd, 'Percentages_4PRISM', 'DCA_'+column+'.csv'), index_col=['dose'])
        ## no duplicate columns - update          
        columns = [c for c in DCA_120.columns.tolist() if c not in d120.columns.tolist()]
        #print('columns', columns)
        if columns != []:
            DCA_120 = pd.concat([DCA_120[columns], d120], axis=1)
            DCA_120.to_csv(os.path.join(cwd, 'Percentages_4PRISM', 'DCA_'+column+'.csv'))
        else: 
            d120.to_csv(os.path.join(cwd, 'Percentages_4PRISM', 'DCA_'+column+'.csv'))    

def pct_unhatched(df1): 
    '''unhatched is computed only over alive embryos'''
    column = 'unhatched120'
    df = df1.copy()
    
    dead = df.groupby(['dose', 'plate_id']).apply(lambda x: float(x['dead120'].sum()))
    c = df.groupby(['dose', 'plate_id']).apply(lambda x: float(x['dead120'].count()))
    
    unhatched = df.groupby(['dose', 'plate_id']).apply(lambda x: float(x['unhatched120'].sum()))
    
    pct = (unhatched/(c-dead))*100
    nSamples = c
    pct = pct.unstack(level=1)
    nSamples = nSamples.unstack(level=1)    
    pct.rename(columns=dict(zip(pct.columns, [col+' - '+column for col in pct.columns])), inplace=True)
    nSamples.rename(columns=dict(zip(nSamples.columns, [col+' - '+'nSamples' for col in nSamples.columns])), inplace=True)
    #print(pct, nSamples)
    return pct, nSamples      

In [167]:
len(c2)
# don't run ctrl plates
c22run = [i for i in c2 if i != '0000']
print(sorted(c22run), len(c22run))
c22run = ['1271'] # <-------------------- run just one compound
c22run = [i for i in c2 if i != '0000'] # beh do all of them

['1036', '1046', '1072', '1111', '1171', '1200', '1229', '1240', '1247', '1271', '1323', '1355', '1409', '1437', '1454', '1477', '1547', '1584', '1623', '1625', '1630', '1656', '1693', '1694', '1719', '1724', '1728', '1747', '1770', '1771', '1794', '1804', '1825', '1895', '1919', '1951', '1956', '1965', '1975', '1978', '1989'] 41


In [168]:
# check if there are ND strings in booleans
booleans = ['bodycurvature', 'snoutjawdefects', 'yolkedema', 'necrosis', 'tailbending', 'notochorddefects'
            , 'craniofacialedema', 'finabsence', 'scoliosis']
for c, csv in enumerate(os.listdir(os.path.join(cwd, 'plate_csvs'))): 
    check_df = pd.read_csv(os.path.join(os.path.join(cwd, 'plate_csvs', csv)))
    for b in booleans: 
        if 'ND' in check_df[b].unique(): 
            print(csv, b)

In [ ]:
# # fix 20220110_1271_DS_R5 that was wrongly named 1217 and that names were in compound column still
# df_batchfile = pd.read_csv(os.path.join(cwd, 'plate_csvs', '20220110_1271_DS_R5.csv'))
# df_batchfile.loc[df.compound == '1217', 'compound'] = '1271'
# df_batchfile.head()
# df_batchfile.to_csv(os.path.join(cwd, 'plate_csvs', '20220110_1271_DS_R5.csv'))

# # fix outfile csv - fix xlxs in sqush to format: 
# df_batchfile = pd.read_csv(os.path.join(cwd, 'Percentages_4PRISM', 'DATA','20221027_BAT2_DS_batch1.csv'))
# df_batchfile.head()
# print(df_batchfile.compound.unique())
# df_batchfile.loc[df_batchfile['compound']=='1217', 'compound'] = '1271'
# print(df_batchfile.compound.unique())
# df_batchfile.head()
# print(cwd)
# print(df_batchfile.columns)
# df_batchfile.to_csv(os.path.join(cwd, 'Percentages_4PRISM', 'DATA','20221027_BAT2_DS_batch1.csv'))
# df_batchfile.to_csv(os.path.join(cwd, 'Percentages_4PRISM','20221027_BAT2_DS_batch1.csv'))

In [169]:
#c22run_subset = ['1036']

for c in c22run: 
#for c in c22run_subset:
    print('COMPOUND', c)
    a = fetch_csvs_byCompound(c, df)
    by_exposure = select_by_exposure(a)
    

    for e, edf in enumerate(by_exposure): # by exposure - chorion  COMPOUND
        exposure = edf['exposure_chorion'].unique()[0]
        print(exposure, edf['plate_id'].unique())
        uni = edf.copy()
        uni = df_format(uni) # format
           
        foo = uni[uni['compound']==c].copy() 
        dmso = uni[uni['compound']=='DMSO'].copy()
        dca = uni[uni['compound']=='DCA'].copy()
        
        phenotype_dfs = []
        dsmo_pheno = []
        
        ### PHENOTYPES ###
        for p, pheno in enumerate(booleans+measured_bool): # by phenotype
            #print(pheno)
            pct, nSamples = get_percentage_nSamples(foo, pheno)
            if phenotype_dfs == []:
                phenotype_dfs.append(nSamples)
                phenotype_dfs.append(pct)
            else: 
                phenotype_dfs.append(pct)
        
        ## merge and save with compound and exposure in filename
        merged = pd.concat(phenotype_dfs, axis=1)
        
        # DMSO phenotypes
        for p, pheno in enumerate(booleans+measured_bool): # by phenotype
            #print(pheno)
            pct, nSamples = get_percentage_nSamples(dmso, pheno)
            if dsmo_pheno == []:
                dsmo_pheno.append(nSamples)
                dsmo_pheno.append(pct)
            else: 
                dsmo_pheno.append(pct)
        merged_dmso = pd.concat(dsmo_pheno, axis=1)
        
        merged = merged.reindex(merged.index.values.tolist()+['DMSO'])
        p = pd.concat([merged, merged_dmso], sort=False)
        #p.to_csv(os.path.join(cwd, 'Percentages_4PRISM', c+'_'+exposure+'_pheno.csv'))
        #print(c+'_'+exposure+'_pheno.csv')
        with pd.ExcelWriter(os.path.join(cwd, 'Percentages_4PRISM', c+'_'+exposure+'_pheno.xlsx')) as writer:  
            p.to_excel(writer, sheet_name=str(c), index=True)
                
        ### MORTALITY ###
        mort_cpd = []
        mort_dmso = []
        for p, pheno in enumerate(dead_unhatched): # by mortality COMPOUND
            #print(pheno)
            if pheno != 'unhatched120': 
                pct, nSamples = get_percentage_nSamples(foo, pheno)
                if mort_cpd == []:
                    mort_cpd.append(nSamples)
                    mort_cpd.append(pct)
                else: 
                    mort_cpd.append(pct)
            else: 
                pct, nSamples = pct_unhatched(foo)
                if mort_cpd == []:
                    mort_cpd.append(nSamples)
                    mort_cpd.append(pct)
                else: 
                    mort_cpd.append(pct)       
                
        cpd_dead = pd.concat(mort_cpd, axis=1)
        
        for p, pheno in enumerate(dead_unhatched): # by mortality DMSO
            #print(pheno)
            if pheno != 'unhatched120': 
                pct, nSamples = get_percentage_nSamples(dmso, pheno)
                if mort_dmso == []:
                    mort_dmso.append(nSamples)
                    mort_dmso.append(pct)
                else: 
                    mort_dmso.append(pct)  
            else: 
                pct, nSamples = pct_unhatched(dmso)
                if mort_dmso == []:
                    mort_dmso.append(nSamples)
                    mort_dmso.append(pct)
                else: 
                    mort_dmso.append(pct)  
        dmso_dead = pd.concat(mort_dmso, axis=1)
        
        cpd_dead = cpd_dead.reindex(cpd_dead.index.values.tolist()+['DMSO'])
        m = pd.concat([cpd_dead, dmso_dead], sort=False)  
        #m.to_csv(os.path.join(cwd, 'Percentages_4PRISM', c+'_'+exposure+'_mort.csv'))
        #print(c+'_'+exposure+'_mort.csv')
        with pd.ExcelWriter(os.path.join(cwd, 'Percentages_4PRISM', c+'_'+exposure+'_mort.xlsx')) as writer:  
            m.to_excel(writer, sheet_name=str(c), index=True)
        
        ## DCA
        mort_dca(dca, 'dead120')
        mort_dca(dca, 'dead24')
        print('---')
        

COMPOUND 1036
found files: 
 58     20220426_1036_DM_R1.csv
63     20220524_1036_DM_R5.csv
105    20220801_1036_DM_R6.csv
Name: filenames, dtype: object
multiple_dose_dechorionated ['20220426_1036_DM_R1' '20220524_1036_DM_R5' '20220801_1036_DM_R6']
---
COMPOUND 1046
found files: 
 0     20220326_1046_DM_R1.csv
5     20220328_1046_DM_R2.csv
10    20220329_1046_DM_R3.csv
Name: filenames, dtype: object
multiple_dose_dechorionated ['20220326_1046_DM_R1' '20220328_1046_DM_R2' '20220329_1046_DM_R3']
---
COMPOUND 1072
found files: 
 89     20220614_1072_DM_R2.csv
100    20220730_1072_DM_R4.csv
106    20220801_1072_DM_R5.csv
Name: filenames, dtype: object
multiple_dose_dechorionated ['20220614_1072_DM_R2' '20220730_1072_DM_R4' '20220801_1072_DM_R5']
---
COMPOUND 1111
found files: 
 59     20220426_1111_DM_R1.csv
64     20220524_1111_DM_R5.csv
112    20220806_1111_DM_R6.csv
Name: filenames, dtype: object
multiple_dose_dechorionated ['20220426_1111_DM_R1' '20220524_1111_DM_R5' '20220806_1111_DM_

### if we had positive control plates, then would need to process them here.... (check BAT2 2019 notebooks)

In [170]:
# merge excel files to check that content is allright
xls = []
for fil in os.listdir(os.path.join(cwd, 'Percentages_4PRISM')): 
    if 'multiple' in fil or 'single' in fil: 
        print(fil)
        df = pd.read_excel(os.path.join(cwd, 'Percentages_4PRISM', fil))
        xls.append(df)

1036_single_dose_dechorionated_pheno.xlsx
1036_single_dose_dechorionated_mort.xlsx
1046_single_dose_dechorionated_pheno.xlsx
1046_single_dose_dechorionated_mort.xlsx
1072_single_dose_dechorionated_pheno.xlsx
1072_single_dose_dechorionated_mort.xlsx
1111_single_dose_dechorionated_pheno.xlsx
1111_single_dose_dechorionated_mort.xlsx
1171_single_dose_dechorionated_pheno.xlsx
1171_single_dose_dechorionated_mort.xlsx
1200_single_dose_dechorionated_pheno.xlsx
1200_single_dose_dechorionated_mort.xlsx
1229_single_dose_dechorionated_pheno.xlsx
1229_single_dose_dechorionated_mort.xlsx
1240_single_dose_dechorionated_pheno.xlsx
1240_single_dose_dechorionated_mort.xlsx
1247_single_dose_dechorionated_pheno.xlsx
1247_single_dose_dechorionated_mort.xlsx
1271_single_dose_dechorionated_pheno.xlsx
1271_single_dose_dechorionated_mort.xlsx
1323_single_dose_dechorionated_pheno.xlsx
1323_single_dose_dechorionated_mort.xlsx
1355_single_dose_dechorionated_pheno.xlsx
1355_single_dose_dechorionated_mort.xlsx
1409

In [171]:
# check if there is some problem with xlsx data - whole column empty...

xls_merge = xls[0].copy()
for i, item in enumerate(xls):
    for col in item.columns:
        #print(col)
        if list(item[col].unique()) == [np.nan]: 
            print('meeep:', col, 'is empty')

# ---------------------- #
# MUNCH a number of (x) compounds to BAT2 format

here we'll merge all 40 - except for the compound that has to be completed ['1547']...

- batch 1 ['1036', '1046', '1072', '1111', '1171', '1200', '1229', '1240', '1247', '1271', '1323', '1355', '1409', '1437', '1454', '1477', '1584', '1623', '1625', '1630', '1656', '1693', '1694', '1719', '1724', '1728', '1747', '1770', '1771', '1794', '1804', '1825', '1895', '1919', '1951', '1956', '1965', '1975', '1978', '1989']


## AAAAND it has been done - compound ['1547'] has been completed 

In [172]:
# DM DS is selected above...
print(df_platefolders[df_platefolders['condition']=='DM'].shape)
print(df_platefolders.shape)

(123, 6)
(123, 6)


In [173]:
# select a batch of compounds
cpd_batch = c2

dfs2go = []
# prepend if pos control 1804....

# if '1804' in cpd_batch: 
#     d = pd.read_csv(os.path.join(cwd, 'plate_csvs', '20201102_0000_CS_R4.csv'), index_col=0)
#     dfs2go.append(d)
      
for comp in cpd_batch: 
    print(comp)
    fils = df_platefolders[df_platefolders['compound']==comp]['filenames']
    for f in fils: 
        print(f)
        d = pd.read_csv(os.path.join(cwd, 'plate_csvs', f), index_col=0)
        dfs2go.append(d)
        
# # append if pos control 1794....
# if '1794' in cpd_batch: 
#     d = pd.read_csv(os.path.join(cwd, 'plate_csvs', '20201103_0000_CS_R5.csv'), index_col=0)
#     dfs2go.append(d)
    
alldfs = pd.concat(dfs2go)
print(alldfs.shape)
alldfs.reset_index(inplace=True)
alldfs.tail()

1036
20220426_1036_DM_R1.csv
20220524_1036_DM_R5.csv
20220801_1036_DM_R6.csv
1046
20220326_1046_DM_R1.csv
20220328_1046_DM_R2.csv
20220329_1046_DM_R3.csv
1072
20220614_1072_DM_R2.csv
20220730_1072_DM_R4.csv
20220801_1072_DM_R5.csv
1111
20220426_1111_DM_R1.csv
20220524_1111_DM_R5.csv
20220806_1111_DM_R6.csv
1171
20220326_1171_DM_R1.csv
20220328_1171_DM_R2.csv
20220329_1171_DM_R3.csv
1200
20220613_1200_DM_R1.csv
20220614_1200_DM_R2.csv
20220727_1200_DM_R4.csv
1229
20220730_1229_DM_R5.csv
20220801_1229_DM_R6.csv
20221008_1229_DM_R8.csv
1240
20220530_1240_DM_R2.csv
20220531_1240_DM_R3.csv
20220725_1240_DM_R5.csv
1247
20220528_1247_DM_R1.csv
20220530_1247_DM_R2.csv
20220531_1247_DM_R3.csv
1271
20220402_1271_DM_R1.csv
20220404_1271_DM_R2.csv
20220810_1271_DM_R6.csv
1323
20220528_1323_DM_R1.csv
20220531_1323_DM_R3.csv
20220806_1323_DM_R4.csv
1355
20220426_1355_DM_R1.csv
20220524_1355_DM_R5.csv
20220806_1355_DM_R6.csv
1409
20220607_1409_DM_R3.csv
20220730_1409_DM_R6.csv
20220810_1409_DM_R7.csv

,plate_name,well_name,chorion,exposure,compound,dose,unit,dead24,dead120,unhatched120,creation_date,well_folder,lateral_image,dorsal_image,show2user,excluded,correction_done,bodycurvature,snoutjawdefects,yolkedema,necrosis,tailbending,notochorddefects,craniofacialedema,finabsence,scoliosis,fishoutline_dorsal_image_area,fishoutline_lateral_image_area,fishoutline_unit,eyes_dorsal_image_eye_down_length,eyes_dorsal_image_eye_up_length,pigmentation_dorsal_image_area_fish_wo_eyes,pigmentation_dorsal_image_area_pigmentation,pigmentation_dorsal_image_area_ratio,pigmentation_unit,pigmentation_threshold,heart_lateral_image_area,heart_unit,bodylength_lateral_image_length,bodylength_unit,condition,avg_eye_diameter,Craniofacial: eye defects,Trunk: short,Trunk: long,Percardial tissue: Edema,Pigment: abnormal,Pigment: decreased,Pigment: absent coloration
11803,20220802_1989_DM_R4,H08,dechorionated,multiple_dose,DMSO,0.5,%,0.0,0,0,2022-08-02T08:55:13.789917+02:00[Europe/Paris],Well_H08,20220802_1989_DM_R4_tail_W_H08_1_2.png,20220802_1989_DM_R4_tail_W_H08_1_1.png,1,False,True,False,False,False,False,False,False,False,False,False,1083524.75,1413189.25,micron_square,365.16,363.22,912188.75,328938.50,0.36,micron_square,45.0,37147.00,micron_square,3927.31,micron,DMSO_0.5_%,364.190,False,False,False,False,False,False,NaN
11804,20220802_1989_DM_R4,H09,dechorionated,multiple_dose,DMSO,0.5,%,0.0,0,0,2022-08-02T09:01:02.266500+02:00[Europe/Paris],Well_H09,20220802_1989_DM_R4_tail_W_H09_1_2.png,20220802_1989_DM_R4_tail_W_H09_1_1.png,1,False,True,False,False,False,False,False,False,False,False,False,1092690.50,1415004.25,micron_square,372.31,371.74,919690.75,341008.25,0.37,micron_square,45.0,34515.25,micron_square,3828.04,micron,DMSO_0.5_%,372.025,False,False,False,False,False,False,NaN
11805,20220802_1989_DM_R4,H10,dechorionated,multiple_dose,DMSO,0.5,%,0.0,0,0,2022-08-02T09:07:58.024459+02:00[Europe/Paris],Well_H10,20220802_1989_DM_R4_head_W_H10_1_4.png,20220802_1989_DM_R4_head_W_H10_1_1.png,1,False,True,False,False,False,False,False,False,False,False,False,1176815.75,1513044.50,micron_square,383.07,382.94,993289.00,411339.50,0.41,micron_square,45.0,34817.75,micron_square,4064.50,micron,DMSO_0.5_%,383.005,False,False,False,False,False,False,NaN
11806,20220802_1989_DM_R4,H11,dechorionated,multiple_dose,DMSO,0.5,%,0.0,0,0,2022-08-02T09:14:59.119361+02:00[Europe/Paris],Well_H11,20220802_1989_DM_R4_head_W_H11_1_4.png,20220802_1989_DM_R4_head_W_H11_1_1.png,1,False,True,False,False,False,False,False,False,False,False,False,1182654.00,1468123.25,micron_square,395.25,407.31,987874.25,443616.25,0.45,micron_square,45.0,49398.25,micron_square,3745.54,micron,DMSO_0.5_%,401.280,False,False,False,False,False,False,NaN
11807,20220802_1989_DM_R4,H12,dechorionated,multiple_dose,DCA,0.0,uM,0.0,0,0,2022-08-02T09:20:57.614271+02:00[Europe/Paris],Well_H12,20220802_1989_DM_R4_head_W_H12_1_4.png,20220802_1989_DM_R4_head_W_H12_1_1.png,1,False,True,False,False,False,False,False,False,False,False,False,1261909.00,1572879.00,micron_square,386.13,386.34,1075448.00,483576.50,0.45,micron_square,45.0,48097.50,micron_square,4108.59,micron,DCA_0.0_uM,386.235,False,False,False,False,True,False,NaN


In [174]:
# for some reason there is an almost empty column plate_name...
alldfs.plate_name.unique()

array(['20220426_1036_DM_R1', '20220524_1036_DM_R5',
       '20220801_1036_DM_R6', '20220326_1046_DM_R1',
       '20220328_1046_DM_R2', '20220329_1046_DM_R3',
       '20220614_1072_DM_R2', '20220730_1072_DM_R4',
       '20220801_1072_DM_R5', '20220426_1111_DM_R1',
       '20220524_1111_DM_R5', '20220806_1111_DM_R6',
       '20220326_1171_DM_R1', '20220328_1171_DM_R2',
       '20220329_1171_DM_R3', '20220613_1200_DM_R1',
       '20220614_1200_DM_R2', '20220727_1200_DM_R4',
       '20220730_1229_DM_R5', '20220801_1229_DM_R6',
       '20221008_1229_DM_R8', '20220530_1240_DM_R2',
       '20220531_1240_DM_R3', '20220725_1240_DM_R5',
       '20220528_1247_DM_R1', '20220530_1247_DM_R2',
       '20220531_1247_DM_R3', '20220402_1271_DM_R1',
       '20220404_1271_DM_R2', '20220810_1271_DM_R6',
       '20220528_1323_DM_R1', '20220531_1323_DM_R3',
       '20220806_1323_DM_R4', '20220426_1355_DM_R1',
       '20220524_1355_DM_R5', '20220806_1355_DM_R6',
       '20220607_1409_DM_R3', '20220730_1409_D

In [175]:
def string_replace(df): 
    for col in df.columns: 
        df[col] = df[col].replace('False', 0, regex=True)
        df[col] = df[col].replace('True', 1, regex=True)
        df[col] = df[col].replace('None', np.nan, regex=True)
    return df
#string replace
alldfs = string_replace(alldfs)

def set_pos_control(value): 
    if value == 'DCA': 
        return 1
    else:  
        return 0
def set_neg_control(value): 
    if value == 'DMSO': 
        return 1
    else: 
        return 0

# additional columns
#alldfs['Is_Vehicle_Control'] = alldfs['compound'].map(lambda x: 1 if x=='DMSO' else 0)
#alldfs['Is_Positive_Control'] = alldfs['compound'].map(lambda x: 1 if x=='DCA' else 0)

alldfs['Is_Vehicle_Control'] = alldfs['compound'].map(lambda x: set_neg_control(x))
alldfs['Is_Positive_Control'] = alldfs['compound'].map(lambda x: set_pos_control(x))

alldfs['Craniofacial: snout defects'] = alldfs['snoutjawdefects']
alldfs['Craniofacial: edema'] = alldfs['craniofacialedema']
alldfs['Yolk sac: Edema'] = alldfs['yolkedema']

alldfs = alldfs[['plate_name', 'well_name', 'chorion', 'exposure', 'compound', 'dose', 'unit'
              , 'Is_Vehicle_Control', 'Is_Positive_Control', 'dead24', 'dead120', 'unhatched120'
              , 'creation_date', 'well_folder', 'lateral_image', 'dorsal_image'
              , 'show2user', 'excluded', 'correction_done'
              , 'bodycurvature', 'snoutjawdefects', 'yolkedema', 'necrosis', 'tailbending'
              , 'notochorddefects', 'craniofacialedema', 'finabsence', 'scoliosis'
              , 'fishoutline_dorsal_image_area', 'fishoutline_lateral_image_area', 'fishoutline_unit'
              , 'eyes_dorsal_image_eye_down_length', 'eyes_dorsal_image_eye_up_length', 'avg_eye_diameter'
              , 'pigmentation_dorsal_image_area_fish_wo_eyes', 'pigmentation_dorsal_image_area_pigmentation'
              , 'pigmentation_dorsal_image_area_ratio', 'pigmentation_unit', 'pigmentation_threshold'
              , 'heart_lateral_image_area', 'heart_unit'
              , 'bodylength_lateral_image_length', 'bodylength_unit'
              , 'Craniofacial: eye defects'
              , 'Craniofacial: snout defects'
              , 'Craniofacial: edema'
              , 'Yolk sac: Edema'
              , 'Trunk: short'
              , 'Trunk: long'
              , 'Percardial tissue: Edema'
              , 'Pigment: abnormal'
              , 'Pigment: decreased'
              , 'Pigment: absent coloration']].copy()

alldfs['Craniofacial: snout defects'] = alldfs['Craniofacial: snout defects'].replace(0.0, False, regex=True)
alldfs['Craniofacial: snout defects'] = alldfs['Craniofacial: snout defects'].replace(1.0, True, regex=True)
alldfs['Yolk sac: Edema'] = alldfs['Yolk sac: Edema'].replace(0.0, False, regex=True)
alldfs['Yolk sac: Edema'] = alldfs['Yolk sac: Edema'].replace(1.0, True, regex=True)
alldfs['Craniofacial: edema'] = alldfs['Craniofacial: edema'].replace(0, False, regex=True)
alldfs['Craniofacial: edema'] = alldfs['Craniofacial: edema'].replace(1.0, True, regex=True)

for col in ['bodycurvature', 'snoutjawdefects', 'yolkedema', 'necrosis', 'tailbending', 'notochorddefects', 'craniofacialedema', 'finabsence', 'scoliosis']: 
    alldfs[col] = alldfs[col].replace(0.0, False, regex=True)
    alldfs[col] = alldfs[col].replace(1.0, True, regex=True)
alldfs.tail()

,plate_name,well_name,chorion,exposure,compound,dose,unit,Is_Vehicle_Control,Is_Positive_Control,dead24,dead120,unhatched120,creation_date,well_folder,lateral_image,dorsal_image,show2user,excluded,correction_done,bodycurvature,snoutjawdefects,yolkedema,necrosis,tailbending,notochorddefects,craniofacialedema,finabsence,scoliosis,fishoutline_dorsal_image_area,fishoutline_lateral_image_area,fishoutline_unit,eyes_dorsal_image_eye_down_length,eyes_dorsal_image_eye_up_length,avg_eye_diameter,pigmentation_dorsal_image_area_fish_wo_eyes,pigmentation_dorsal_image_area_pigmentation,pigmentation_dorsal_image_area_ratio,pigmentation_unit,pigmentation_threshold,heart_lateral_image_area,heart_unit,bodylength_lateral_image_length,bodylength_unit,Craniofacial: eye defects,Craniofacial: snout defects,Craniofacial: edema,Yolk sac: Edema,Trunk: short,Trunk: long,Percardial tissue: Edema,Pigment: abnormal,Pigment: decreased,Pigment: absent coloration
11803,20220802_1989_DM_R4,H08,dechorionated,multiple_dose,DMSO,0.5,%,1,0,0.0,0,0,2022-08-02T08:55:13.789917+02:00[Europe/Paris],Well_H08,20220802_1989_DM_R4_tail_W_H08_1_2.png,20220802_1989_DM_R4_tail_W_H08_1_1.png,1,False,True,False,False,False,False,False,False,False,False,False,1083524.75,1413189.25,micron_square,365.16,363.22,364.190,912188.75,328938.50,0.36,micron_square,45.0,37147.00,micron_square,3927.31,micron,False,False,False,False,False,False,False,False,False,NaN
11804,20220802_1989_DM_R4,H09,dechorionated,multiple_dose,DMSO,0.5,%,1,0,0.0,0,0,2022-08-02T09:01:02.266500+02:00[Europe/Paris],Well_H09,20220802_1989_DM_R4_tail_W_H09_1_2.png,20220802_1989_DM_R4_tail_W_H09_1_1.png,1,False,True,False,False,False,False,False,False,False,False,False,1092690.50,1415004.25,micron_square,372.31,371.74,372.025,919690.75,341008.25,0.37,micron_square,45.0,34515.25,micron_square,3828.04,micron,False,False,False,False,False,False,False,False,False,NaN
11805,20220802_1989_DM_R4,H10,dechorionated,multiple_dose,DMSO,0.5,%,1,0,0.0,0,0,2022-08-02T09:07:58.024459+02:00[Europe/Paris],Well_H10,20220802_1989_DM_R4_head_W_H10_1_4.png,20220802_1989_DM_R4_head_W_H10_1_1.png,1,False,True,False,False,False,False,False,False,False,False,False,1176815.75,1513044.50,micron_square,383.07,382.94,383.005,993289.00,411339.50,0.41,micron_square,45.0,34817.75,micron_square,4064.50,micron,False,False,False,False,False,False,False,False,False,NaN
11806,20220802_1989_DM_R4,H11,dechorionated,multiple_dose,DMSO,0.5,%,1,0,0.0,0,0,2022-08-02T09:14:59.119361+02:00[Europe/Paris],Well_H11,20220802_1989_DM_R4_head_W_H11_1_4.png,20220802_1989_DM_R4_head_W_H11_1_1.png,1,False,True,False,False,False,False,False,False,False,False,False,1182654.00,1468123.25,micron_square,395.25,407.31,401.280,987874.25,443616.25,0.45,micron_square,45.0,49398.25,micron_square,3745.54,micron,False,False,False,False,False,False,False,False,False,NaN
11807,20220802_1989_DM_R4,H12,dechorionated,multiple_dose,DCA,0.0,uM,0,1,0.0,0,0,2022-08-02T09:20:57.614271+02:00[Europe/Paris],Well_H12,20220802_1989_DM_R4_head_W_H12_1_4.png,20220802_1989_DM_R4_head_W_H12_1_1.png,1,False,True,False,False,False,False,False,False,False,False,False,1261909.00,1572879.00,micron_square,386.13,386.34,386.235,1075448.00,483576.50,0.45,micron_square,45.0,48097.50,micron_square,4108.59,micron,False,False,False,False,False,False,False,True,False,NaN


In [176]:
alldfs.to_csv(os.path.join(cwd, 'Percentages_4PRISM', '20221125_BAT2_DM_batch1.csv'))

In [ ]:
alldfs.columns

# now squash to format